In [1]:
import scipy.linalg as spl

import pennylane as qml
from pennylane import numpy as np

from qbmqsp.hamiltonian import Hamiltonian
from qbmqsp.qsp_phase_engine import QSPPhaseEngine
from qbmqsp.qevt import QEVT

In [2]:
def test_qevt(encoding):
    h = ["ZZII", "IZZI", "IIZZ", "ZIIZ",
         "XIII", "IXII", "IIXI", "IIIX",
         "YIII", "IYII", "IIYI", "IIIY"]

    n_params = len(h)
    n_qubits = len(h[0])
    np.random.seed(seed=1)
    θ = np.random.random(n_params) / n_params

    (β, δ, θ_norm) = 1, 0.3, np.linalg.norm(θ, ord=1)
    τ = β * θ_norm / (1-δ)
    qsp_phase_engine = QSPPhaseEngine(δ, polydeg=100.0)
    φ = qsp_phase_engine.generate(τ)
    del qsp_phase_engine

    h_δ = h + [n_qubits * 'I']
    θ_δ = np.append(θ * (1-δ)/(2*θ_norm), (1+δ)/2)

    qevt = QEVT(h_δ, θ_δ, encoding, φ)
    n_aux, n_enc, n_sys = qevt.n_qubits({'aux'}), qevt.n_qubits({'enc'}), qevt.n_qubits({'sys'})
    wires = list(range(n_aux + n_enc + n_sys))
    aux_wire = wires[: n_aux]
    enc_wires = wires[n_aux : n_aux+n_enc]
    sys_wires = wires[n_aux+n_enc : n_aux+n_enc+n_sys]
    
    dev = qml.device('default.qubit', wires=n_aux+n_enc+n_sys)
    @qml.qnode(dev)
    def qevt_circuit():
        qevt.circuit(aux_wire, enc_wires, sys_wires)
        return qml.state()
    out = qevt_circuit().reshape(2**(n_aux+n_enc), 2**n_sys)[0, :] # = (<00| x I) QEVT |000>
    # out /= np.linalg.norm(out) #normalized

    H = Hamiltonian(h, θ)
    ideal = spl.expm(-β/2*H.assemble())[:, 0] * spl.expm(-β*θ_norm*(1+δ)/(1-δ)/2) #/ np.sqrt(expm(-β*H)[0, 0]) #normalized
    
    error = np.sum(np.abs(out - ideal))
    return error

In [3]:
test_qevt('general')


Starting MATLAB engine.. Done.


Stopping MATLAB engine ... Done.



tensor(0.00084171, requires_grad=True)

In [4]:
test_qevt('lcu')


Starting MATLAB engine.. Done.


Stopping MATLAB engine ... Done.



tensor(0.00084171, requires_grad=True)